<a href="https://colab.research.google.com/github/utsavdutta/recommender_system/blob/master/recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

In [63]:
import numpy as np
from typing import Dict, Text
import tensorflow as tf
import pprint
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [64]:
ratings = tfds.load('movielens/100k-ratings', split="train")
movies = tfds.load('movielens/100k-movies', split="train")

for x in ratings.take(1).as_numpy_iterator():
  pprint.pprint(x);
print();
for x in movies.take(1).as_numpy_iterator():
  pprint.pprint(x);

{'bucketized_user_age': 45.0,
 'movie_genres': array([7]),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}

{'movie_genres': array([4]),
 'movie_id': b'1681',
 'movie_title': b'You So Crazy (1994)'}


In [73]:
tf.random.set_seed(42);
shuffle = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False);

train = shuffle.take(80_000);
test = shuffle.skip(80_000).take(20_000);


# populating the vocabulary for movie_titles and user_ids 
# we need to do this so we can  map the movie_titles and user_ids into embeddings in embedding layer

new_ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
});
new_movies = movies.map(lambda x: x["movie_title"]);

movies_titles = new_movies.batch(1_000);
user_ids = new_ratings.batch(1_000_000).map(lambda x: x["user_id"]);


unique_movie_titles = np.unique(np.concatenate(list(movies_titles)));
unique_user_ids = np.unique(np.concatenate(list(user_ids)));



# unique_movie_titles[:4]



Retreval model

In [66]:
user_model = tf.keras.Sequential([tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary=unique_user_ids, mask_token=None),
tf.keras.layers.Embedding(len(unique_user_ids)+1, 64)]);
movie_model = tf.keras.Sequential([tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary=unique_movie_titles, mask_token=None),
tf.keras.layers.Embedding(len(unique_movie_titles)+1, 64)]);

# FactorizedTopK metric helps evaluate how often the true candidate is in the top k candidates for a given query
metrics = tfrs.metrics.FactorizedTopK(new_movies.batch(128).map(movie_model), k=100); 

# retival task object is a wrapper that bundles together loss fn and metric computation and returns computed loss
task = tfrs.tasks.Retrieval(metrics= metrics);

In [67]:
class MovieLensModel(tfrs.Model):

  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.movie_model = movie_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.


    user_embeddings = self.user_model(features["user_id"])
    positive_movie_embeddings = self.movie_model(features["movie_title"])

    return self.task(user_embeddings, positive_movie_embeddings);

### Fit and evaluate it.

Create the model, train it, and generate predictions:

In [80]:
# Create a retrieval model.
model = MovieLensModel(user_model, movie_model, task);
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5));

model.fit(new_ratings.batch(4096), epochs=3);

# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model);
index.index_from_dataset(new_movies.batch(100).map(lambda title: (title, model.movie_model(title))));

# Get some recommendations.
_, titles = index(np.array(["42"]));
print(f"Top 3 recommendations for user 42: {titles[0, :3]}");


# model = MovieLensModel(user_model, movie_model, task);
# model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5));

# cached_train = train.shuffle(100_000).batch(8192).cache();
# cached_test = test.batch(4096).cache();


# model.fit(cached_train, epochs=2);





Epoch 1/3
25/25 [==============================] - 78s 1s/step - factorized_top_k/top_1_categorical_accuracy: 3.3000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0048 - factorized_top_k/top_10_categorical_accuracy: 0.0122 - factorized_top_k/top_50_categorical_accuracy: 0.0836 - factorized_top_k/top_100_categorical_accuracy: 0.1731 - loss: 33279.8113 - regularization_loss: 0.0000e+00 - total_loss: 33279.8113
Epoch 2/3
25/25 [==============================] - 26s 1s/step - factorized_top_k/top_1_categorical_accuracy: 7.6000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0085 - factorized_top_k/top_10_categorical_accuracy: 0.0218 - factorized_top_k/top_50_categorical_accuracy: 0.1429 - factorized_top_k/top_100_categorical_accuracy: 0.2686 - loss: 30530.9643 - regularization_loss: 0.0000e+00 - total_loss: 30530.9643
Epoch 3/3
25/25 [==============================] - 27s 1s/step - factorized_top_k/top_1_categorical_accuracy: 6.8000e-04 - factorized_top_k/top_5_categorical_accu